In [1]:
!pip install --upgrade -q transformers datasets 
!pip install -q huggingface_hub

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 103.3 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 26.4 MB/s eta 0:00:00


In [2]:
from huggingface_hub import notebook_login
notebook_login()

In [4]:
import transformers
import torch

model_id = "meta-llama/Meta-Llama-3.1-8B-Instruct"

pipeline = transformers.pipeline(
    "text-generation",
    model=model_id,
    model_kwargs={"torch_dtype": torch.bfloat16},
    device_map="auto",
)

prompt = "question: What role does fiscal policy play in economic growth? \n\n Retrieved chunks: ['Fiscal policy, which involves government spending and taxation decisions, plays a significant role in stimulating economic growth during periods of recession. Increased government spending can boost demand, while tax cuts can increase disposable income for consumers, both of which can contribute to a higher rate of economic activity.', 'Economic growth is influenced by a myriad of factors, including technological innovation, consumer confidence, and global trade dynamics. While fiscal policy is a critical tool, its effectiveness can be moderated by other external factors that impact the overall business environment.', 'The impact of fiscal policy is also subject to political considerations, where budget constraints and public debt levels can limit the scope of spending initiatives. Moreover, the multiplier effect of fiscal spending may vary depending on the state of the economy and the efficiency of government programs.']"
messages = [
        {"role": "system", "content": """You are an NLP assistant that processes a user input containing a question and a list of jumbled text chunks retrieved from various sources. Your task is to:

Identify which chunks are useful for answering the given question.
Order the useful chunks in a sequence that best answers the question.
Return your result in JSON format with two fields:
"useful_count": the total number of chunks you identified as relevant.
"ordered_chunks": an array of the selected chunks in the correct order.
Please ignore any chunk that is not relevant to the question.
Example Response Format:
<example>{
"ordered_chunks": [
"Private equity typically involves investing in established companies with the aim of improving operational efficiency and profitability, often through restructuring or strategic acquisitions."
],
"useful_count": 1
} </example>"""},
        {"role": "user", "content": prompt }
    ]

outputs = pipeline(
    messages,
    max_new_tokens=4096,
)
print(outputs[0]["generated_text"][-1])
print('__'*50)
print(outputs)


config.json:   0%|          | 0.00/855 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/55.4k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

Device set to use cuda:0
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


{'role': 'assistant', 'content': 'To identify the useful chunks for answering the question, I will analyze each chunk and determine its relevance to the topic of fiscal policy and economic growth.\n\nChunk 1: This chunk is highly relevant as it directly discusses the role of fiscal policy in stimulating economic growth during periods of recession.\nChunk 2: This chunk is relevant as it mentions that fiscal policy is a critical tool for economic growth, but also notes that its effectiveness can be moderated by other external factors.\nChunk 3: This chunk is relevant as it discusses the impact of fiscal policy, including the limitations imposed by budget constraints and public debt levels.\n\nI will order the chunks in a sequence that best answers the question:\n\n1. Chunk 1: This chunk provides a clear explanation of the role of fiscal policy in stimulating economic growth during periods of recession.\n2. Chunk 2: This chunk provides additional context on the limitations of fiscal polic

# DATASET INFERENCE

In [6]:
from datasets import load_dataset

fin_dataset = load_dataset("AmrataYadav/fin_retrieval_checker_finetune_dataset", split = "test")

README.md:   0%|          | 0.00/730 [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/43.7k [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/20.2k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/32.0k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/44 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/12 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25 [00:00<?, ? examples/s]

In [9]:
from datasets import DatasetDict

def process_dataset(dataset, pipeline, model_column="Llama-3.1-8b-instruct-output"):
    """
    Processes a Hugging Face dataset by running a text generation pipeline and storing the outputs.

    Args:
        dataset (DatasetDict): The dataset containing input prompts.
        pipeline (Callable): The text generation model pipeline.
        model_column (str): The name of the column to store the model outputs.

    Returns:
        DatasetDict: Updated dataset with model-generated responses.
    """
    def generate_output(examples):
        messages = [
            {"role": "system", "content": """You are an NLP assistant that processes a user input containing a question and a list of jumbled text chunks retrieved from various sources. Your task is to:
            Identify which chunks are useful for answering the given question.
            Order the useful chunks in a sequence that best answers the question.
            Return your result in JSON format with two fields:
            "useful_count": the total number of chunks you identified as relevant
            "ordered_chunks": an array of the selected chunks in the correct order.
            Please ignore any chunk that is not relevant to the question.
            Example Response Format:
            <example>{
            "ordered_chunks": [
            "Private equity typically involves investing in established companies with the aim of improving operational efficiency and profitability, often through restructuring or strategic acquisitions."
            ],
            "useful_count": 1
            } </example>"""},
            {"role": "user", "content": str(examples['prompt'])}
        ]
        
        output = pipeline(messages, max_new_tokens=4096,)
        print(f"Input Prompt: {examples['prompt']}")
        print("-" * 50)
        print(f"Generated Output: {output[0]['generated_text'][-1]}")
        print("__" * 50)
        return {model_column: output[0]["generated_text"][-1]}

    return dataset.map(generate_output)

In [ ]:
# Example usage:
updated_dataset = process_dataset(fin_dataset, pipeline)

Parameter 'function'=<function process_dataset.<locals>.generate_output at 0x7fb1817a0dc0> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


Map:   0%|          | 0/25 [00:00<?, ? examples/s]

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Input Prompt: question: What role does fiscal policy play in economic growth? 

 Retrieved chunks: ['Fiscal policy, which involves government spending and taxation decisions, plays a significant role in stimulating economic growth during periods of recession. Increased government spending can boost demand, while tax cuts can increase disposable income for consumers, both of which can contribute to a higher rate of economic activity.', 'Economic growth is influenced by a myriad of factors, including technological innovation, consumer confidence, and global trade dynamics. While fiscal policy is a critical tool, its effectiveness can be moderated by other external factors that impact the overall business environment.', 'The impact of fiscal policy is also subject to political considerations, where budget constraints and public debt levels can limit the scope of spending initiatives. Moreover, the multiplier effect of fiscal spending may vary depending on the state of the economy and the 

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


...

Input Prompt: question: What factors contribute to the pricing of derivatives? 

 Retrieved chunks: ["The pricing of derivatives, such as options and futures, is influenced by factors including the underlying asset's price, volatility, time to expiration, and prevailing interest rates.", 'Mathematical models like the Black-Scholes formula are commonly used to determine the fair value of derivatives based on these inputs, although market conditions can lead to deviations from theoretical values.', 'Market sentiment and supply-demand dynamics also play a role, as investor perceptions and trading volumes can cause derivative prices to diverge from model predictions.']
--------------------------------------------------
Generated Output: {'role': 'assistant', 'content': 'To identify the useful chunks for answering the given question, I will analyze each chunk and determine its relevance to the question.\n\nChunk 1: "The pricing of derivatives, such as options and futures, is influenced by f

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Input Prompt: question: What is the impact of currency appreciation on export competitiveness? 

 Retrieved chunks: ["Currency appreciation makes exports more expensive, reducing a country's competitiveness in international markets.", 'The balance of payments can be influenced by fluctuations in currency values.', 'Fiscal deficits have a long-term effect on government borrowing costs.']
--------------------------------------------------
Generated Output: {'role': 'assistant', 'content': 'To determine the relevant chunks for the given question, I will analyze each chunk and identify its relevance to the topic of currency appreciation and export competitiveness.\n\nChunk 1: "Currency appreciation makes exports more expensive, reducing a country\'s competitiveness in international markets."\nRelevance: High - This chunk directly addresses the impact of currency appreciation on export competitiveness.\n\nChunk 2: "The balance of payments can be influenced by fluctuations in currency values

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Input Prompt: question: What are the implications of a flattening yield curve? 

 Retrieved chunks: ['A flattening yield curve occurs when the difference between short-term and long-term interest rates narrows, which can be an indicator of changing economic conditions.', 'Investors often interpret a flattening yield curve as a signal that economic growth may be slowing, potentially leading to reduced profitability in sectors such as banking.', 'However, some economists argue that a flattening curve can also result from central bank policies and does not always predict an imminent recession.']
--------------------------------------------------
Generated Output: {'role': 'assistant', 'content': '{\n  "useful_count": 3,\n  "ordered_chunks": [\n    "A flattening yield curve occurs when the difference between short-term and long-term interest rates narrows, which can be an indicator of changing economic conditions.",\n    "Investors often interpret a flattening yield curve as a signal that 

In [ ]:
updated_dataset.push_to_hub("fin_retrieval_checker_finetune_dataset-llama-3.1-8b-instruct-responses")